# Exercise 6

## Predict rating using LSTM


In [ ]:
import pandas as pd

In [ ]:
#Se extrae una base de entrenamiento de un csv en una ruta de github
dataTraining = pd.read_csv('https://github.com/sergiomora03/AdvancedTopicsAnalytics/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

In [ ]:
plots = dataTraining['plot']
# Se crea una variable dummy donde true indica que la calificación es mayor o igual que la media y los valores False indican que la calificación es menor que la media.
y = (dataTraining['rating'] >= dataTraining['rating'].mean()).astype(int)

In [ ]:
plots

3107    most is the story of a single father who takes...
900     a serial killer decides to teach the secrets o...
6724    in sweden ,  a female blackmailer with a disfi...
4704    in a friday afternoon in new york ,  the presi...
2582    in los angeles ,  the editor of a publishing h...
                              ...                        
8417    " our marriage ,  their wedding .  "  it ' s l...
1592    the wandering barbarian ,  conan ,  alongside ...
1723    like a tale spun by scheherazade ,  kismet fol...
7605    mrs .  brisby ,  a widowed mouse ,  lives in a...
215     tinker bell journey far north of never land to...
Name: plot, Length: 7895, dtype: object

In [ ]:
y

3107    1
900     0
6724    1
4704    1
2582    1
       ..
8417    0
1592    0
1723    0
7605    1
215     1
Name: rating, Length: 7895, dtype: int64

# Exercise 6.1

- Remove stopwords
- Lowercase
- split the text in words
- pad_sequences

In [ ]:
!pip install wget
!pip install livelossplot --quiet

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a01c4561678084acd51519c4b59a62af6d3eddae283be9cac4a32906ac8bdcc4
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import pandas as pd
import numpy as np
import wget
import os
from zipfile import ZipFile

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string

#Se importan diversas librerías y clases necesarias para construir y entrenar modelos de redes neuronales en Keras, así como para realizar visualizaciones y trabajar con modelos de procesamiento de lenguaje natural (NLP) utilizando gensim

from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.utils import pad_sequences
from livelossplot import PlotLossesKeras

import matplotlib.pyplot as plt

import gensim
from gensim.models import Word2Vec
import warnings


warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
X = plots

Se realiza el preprocesamiento de los datos como: Convertir el texto a minúsculas utilizando text.lower(). Eliminar los signos de puntuación del texto utilizando una comprensión de lista y la función string.punctuation. Tokenizar el texto en palabras utilizando word_tokenize del módulo NLTK. Eliminar stopwords en ingles Unir las palabras procesadas en una cadena de texto utilizando ' '.join(tokens) y devolverla como resultado.

In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X = X.apply(preprocess)

In [ ]:
X = X.tolist()


In [ ]:
# construye un vocabulario a partir de los caracteres presentes en la trama
voc = set(''.join(X))#connjunto de caracteres únicos a partir de la cadena de caracteres resultante.
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}#crea un diccionario de comprensión que asigna a cada carácter único en el conjunto un índice único más uno.

In [ ]:
#conversión de caracteres a números y el relleno (padding) de secuencias.
# Max len
max_len = 200 #Máximo deseado para las secuencias de texto.
X = [x[:max_len] for x in X] # Asegura que cada texto en la lista X tenga como máximo una longitud de 200 caracteres.
# Convert characters to int and pad
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in X]#: Este código convierte cada carácter en los textos de X en un número utilizando un vocabulario predefinido (vocabulary).


In [ ]:
X_pad = pad_sequences(X, maxlen=max_len)#aplica el relleno (padding) a las secuencias de entrada

# Exercise 6.2

Create a SimpleRNN neural network to predict the rating of a movie

Calculate the testing set accuracy

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Suponiendo que tienes X y y preprocesados y listos
# X_pad es el resultado del padding que ya has aplicado
# y es la variable binaria a predecir

# Convertir y en formato numpy
y = np.array(y)

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Definir la arquitectura del modelo
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_len)) #Capa embedding para procesar secuencias de texto con una dimensión de el largo del vocabulario mas 1, una dimensión del vector de 128 palabras y una longitud máxima de las secuencias de entrada por el largo del vocaboluario
model.add(SimpleRNN(64))  # Número de unidades en la capa SimpleRNN que permite al modelo aprender y capturar dependencias temporales en los datos de entrada, como secuencias de texto o series temporales.
model.add(Dense(1, activation='sigmoid')) #Función de activación sigmoide

# Compilar el modelo: Se mide el modelo de acuerdo a la minimización de perdida con la cosentropia binaria y maximizando el accuracy:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Calcular la precisión en el conjunto de pruebas
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Testing Set Accuracy:", test_accuracy)

Epoch 1/10
178/178 [==============================] - 17s 83ms/step - loss: 0.6995 - accuracy: 0.4998 - val_loss: 0.7003 - val_accuracy: 0.4763
Epoch 2/10
178/178 [==============================] - 13s 72ms/step - loss: 0.6912 - accuracy: 0.5387 - val_loss: 0.7067 - val_accuracy: 0.4921
Epoch 3/10
178/178 [==============================] - 12s 70ms/step - loss: 0.6918 - accuracy: 0.5318 - val_loss: 0.7021 - val_accuracy: 0.4921
Epoch 4/10
178/178 [==============================] - 13s 71ms/step - loss: 0.6926 - accuracy: 0.5234 - val_loss: 0.7021 - val_accuracy: 0.5063
Epoch 5/10
178/178 [==============================] - 13s 70ms/step - loss: 0.6900 - accuracy: 0.5368 - val_loss: 0.7020 - val_accuracy: 0.4794
Epoch 6/10
178/178 [==============================] - 13s 71ms/step - loss: 0.6882 - accuracy: 0.5449 - val_loss: 0.7062 - val_accuracy: 0.5095
Epoch 7/10
178/178 [==============================] - 13s 73ms/step - loss: 0.6875 - accuracy: 0.5523 - val_loss: 0.7047 - val_accuracy:

Se llega a un accuracy del 51,4%, indicando que el modelo sale igual a predecir aleatoriamente las categorias

# Exercise 6.3

Create a LSTM neural network to predict the rating of a movie

Calculate the testing set accuracy

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Embedding
# Definir la arquitectura del modelo
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_len)) #Capa embedding para procesar secuencias de texto con una dimensión de el largo del vocabulario mas 1, una dimensión del vector de 128 palabras y una longitud máxima de las secuencias de entrada por el largo del vocaboluario
model.add(LSTM(64))  # Número de unidades en la capa LSTM que permite al modelo aprender y capturar dependencias a largo plazo en los datos de entrada, como secuencias de texto o series temporales
model.add(Dense(1, activation='sigmoid')) #Función de activación sigmoide

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Calcular la precisión en el conjunto de pruebas
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Testing Set Accuracy:", test_accuracy)

Epoch 1/10
178/178 [==============================] - 33s 168ms/step - loss: 0.6930 - accuracy: 0.5127 - val_loss: 0.6933 - val_accuracy: 0.5301
Epoch 2/10
178/178 [==============================] - 28s 158ms/step - loss: 0.6905 - accuracy: 0.5267 - val_loss: 0.6970 - val_accuracy: 0.4652
Epoch 3/10
178/178 [==============================] - 27s 155ms/step - loss: 0.6890 - accuracy: 0.5303 - val_loss: 0.6974 - val_accuracy: 0.4858
Epoch 4/10
178/178 [==============================] - 29s 162ms/step - loss: 0.6884 - accuracy: 0.5452 - val_loss: 0.6983 - val_accuracy: 0.4984
Epoch 5/10
178/178 [==============================] - 28s 156ms/step - loss: 0.6872 - accuracy: 0.5466 - val_loss: 0.6981 - val_accuracy: 0.4826
Epoch 6/10
178/178 [==============================] - 28s 155ms/step - loss: 0.6858 - accuracy: 0.5533 - val_loss: 0.7055 - val_accuracy: 0.4810
Epoch 7/10
178/178 [==============================] - 28s 155ms/step - loss: 0.6843 - accuracy: 0.5563 - val_loss: 0.7003 - val_ac

Este modelo secuencial genera una mejor medida de accuracy con respecto al modelo simple RNN

# Exercise 6.4

Create a GRU neural network to predict the rating of a movie

Calculate the testing set accuracy

In [ ]:

from tensorflow.keras.layers import GRU, Dense, Embedding

# Definir la arquitectura del modelo
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_len)) #Capa embedding para procesar secuencias de texto con una dimensión de el largo del vocabulario mas 1, una dimensión del vector de 128 palabras y una longitud máxima de las secuencias de entrada por el largo del vocaboluario
model.add(GRU(64))  # Número de unidades en la capa GRU (Gated Recurrent Unit), lo que permite al modelo aprender y capturar dependencias en los datos de entrada, como secuencias de texto o series temporales.
model.add(Dense(1, activation='sigmoid')) #Función de activación sigmoide

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Calcular la precisión en el conjunto de pruebas
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Testing Set Accuracy:", test_accuracy)

Epoch 1/10
178/178 [==============================] - 28s 141ms/step - loss: 0.6927 - accuracy: 0.5179 - val_loss: 0.6938 - val_accuracy: 0.5316
Epoch 2/10
178/178 [==============================] - 24s 136ms/step - loss: 0.6909 - accuracy: 0.5262 - val_loss: 0.6967 - val_accuracy: 0.5095
Epoch 3/10
178/178 [==============================] - 25s 138ms/step - loss: 0.6900 - accuracy: 0.5311 - val_loss: 0.6982 - val_accuracy: 0.4968
Epoch 4/10
178/178 [==============================] - 26s 146ms/step - loss: 0.6890 - accuracy: 0.5384 - val_loss: 0.6981 - val_accuracy: 0.5111
Epoch 5/10
178/178 [==============================] - 26s 147ms/step - loss: 0.6877 - accuracy: 0.5475 - val_loss: 0.7000 - val_accuracy: 0.4826
Epoch 6/10
178/178 [==============================] - 25s 139ms/step - loss: 0.6873 - accuracy: 0.5417 - val_loss: 0.6982 - val_accuracy: 0.5269
Epoch 7/10
178/178 [==============================] - 25s 140ms/step - loss: 0.6861 - accuracy: 0.5489 - val_loss: 0.7024 - val_ac

Con la implementación de una capa GRU se evidencia que es mejor que el RNN simple pero es inferior su rendimiento con respecto al modelo con capa LSTM